In [ ]:
!pip install scikit-learn==1.7.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 45.9 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.6.1
    Uninstalling scikit-learn-1.6.1:
      Successfully uninstalled scikit-learn-1.6.1


## PASO 1: INSTALAR E IMPORTAR LIBRERÍAS

In [ ]:
# Primero, nos aseguramos de tener todo lo que necesitamos.
# pandas es para manejar los datos (como si fuera una hoja de cálculo de Excel).
# scikit-learn es la biblioteca principal para hacer Machine Learning en Python.
# joblib nos servirá para guardar nuestro modelo una vez que esté entrenado.

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, classification_report
import joblib

print("Librerías importadas correctamente.")

Librerías importadas correctamente.


## PASO 2: CARGAR Y PREPARAR LOS DATOS

In [ ]:
# Ahora, vamos a cargar los archivos CSV que subiste a Colab.
# También, añadiremos una columna 'wine_type' para saber si un vino es tinto o blanco
# y luego los juntaremos en una sola tabla de datos.

try:
    red_wine = pd.read_csv('/content/winequality-red.csv', sep=';')
    white_wine = pd.read_csv('/content/winequality-white.csv', sep=';')
except FileNotFoundError:
    print("Error: No se encontraron los archivos. Asegúrate de haber subido 'winequality-red.csv' y 'winequality-white.csv' a tu sesión de Colab.")
    exit()

# Agregamos la columna para identificar el tipo
red_wine['wine_type'] = 'red'
white_wine['wine_type'] = 'white'

# Combinamos ambos datasets
wines = pd.concat([red_wine, white_wine])

print("Datos cargados y combinados. Total de filas:", len(wines))
print("Primeras 5 filas del dataset combinado:")
print(wines.head())

Datos cargados y combinados. Total de filas: 6497
Primeras 5 filas del dataset combinado:
   fixed acidity  volatile acidity  citric acid  residual sugar  chlorides  \
0            7.4              0.70         0.00             1.9      0.076   
1            7.8              0.88         0.00             2.6      0.098   
2            7.8              0.76         0.04             2.3      0.092   
3           11.2              0.28         0.56             1.9      0.075   
4            7.4              0.70         0.00             1.9      0.076   

   free sulfur dioxide  total sulfur dioxide  density    pH  sulphates  \
0                 11.0                  34.0   0.9978  3.51       0.56   
1                 25.0                  67.0   0.9968  3.20       0.68   
2                 15.0                  54.0   0.9970  3.26       0.65   
3                 17.0                  60.0   0.9980  3.16       0.58   
4                 11.0                  34.0   0.9978  3.51       0.56 

## PASO 3: INGENIERÍA DE CARACTERÍSTICAS Y PREPROCESAMIENTO

In [ ]:
# El modelo necesita que los datos estén en un formato específico.
# 1. Convertiremos la calidad (que es un número del 1 al 10) en categorías ('baja', 'media', 'alta').
#    Esto convierte nuestro problema en uno de clasificación, que es más fácil de manejar.
# 2. Convertiremos el tipo de vino ('red', 'white') en números (0 y 1), porque los modelos
#    de machine learning solo entienden números.

# 1. Crear la variable objetivo categórica
wines['quality_category'] = wines['quality'].apply(
    lambda x: 'baja' if x <= 5 else ('media' if x <= 7 else 'alta')
)


# 2. Convertir 'wine_type' a formato numérico
wines['wine_type'] = wines['wine_type'].apply(lambda x: 0 if x == 'red' else 1)

print("\nSe ha creado la columna 'quality_category' y se ha transformado 'wine_type'.")
print("Distribución de las nuevas categorías de calidad:")
print(wines['quality_category'].value_counts())


Se ha creado la columna 'quality_category' y se ha transformado 'wine_type'.
Distribución de las nuevas categorías de calidad:
quality_category
media    3915
baja     2384
alta      198
Name: count, dtype: int64


## PASO 4: DEFINIR CARACTERÍSTICAS (X) Y OBJETIVO (y)

In [ ]:
# Vamos a separar nuestra tabla en dos partes:
# X: Las características o "pistas" que el modelo usará para aprender (todas las columnas menos la calidad).
# y: El objetivo que queremos predecir (la 'quality_category').

# X son todas las columnas excepto las de calidad
X = wines.drop(['quality', 'quality_category'], axis=1)

# y es solo la columna que queremos predecir
y = wines['quality_category']

print("\nCaracterísticas (X) para el modelo:")
print(X.head())
print("\nObjetivo (y) a predecir:")
print(y.head())


Características (X) para el modelo:
   fixed acidity  volatile acidity  citric acid  residual sugar  chlorides  \
0            7.4              0.70         0.00             1.9      0.076   
1            7.8              0.88         0.00             2.6      0.098   
2            7.8              0.76         0.04             2.3      0.092   
3           11.2              0.28         0.56             1.9      0.075   
4            7.4              0.70         0.00             1.9      0.076   

   free sulfur dioxide  total sulfur dioxide  density    pH  sulphates  \
0                 11.0                  34.0   0.9978  3.51       0.56   
1                 25.0                  67.0   0.9968  3.20       0.68   
2                 15.0                  54.0   0.9970  3.26       0.65   
3                 17.0                  60.0   0.9980  3.16       0.58   
4                 11.0                  34.0   0.9978  3.51       0.56   

   alcohol  wine_type  
0      9.4          0  
1

## PASO 5: DIVIDIR LOS DATOS PARA ENTRENAMIENTO Y PRUEBA

In [ ]:
# Este es un paso CRUCIAL. No podemos usar todos nuestros datos para entrenar el modelo,
# porque si no, no tendríamos cómo saber si realmente aprendió algo o si solo se memorizó las respuestas.
# Dividimos los datos: 80% para entrenar y 20% para probar.

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(f"\nDatos divididos en:")
print(f"- {len(X_train)} filas para entrenamiento.")
print(f"- {len(X_test)} filas para prueba.")


Datos divididos en:
- 5197 filas para entrenamiento.
- 1300 filas para prueba.


## PASO 5.1: ESCALAR LAS CARACTERÍSTICAS (PREPARACIÓN PARA LA RED NEURONAL)

In [ ]:
# Las Redes Neuronales son sensibles a la escala de los datos. Si una característica
# tiene un rango mucho más grande que otras (ej. 'total sulfur dioxide' vs 'pH'),
# puede dominar el proceso de aprendizaje. Por eso, estandarizamos los datos.
# El StandardScaler transforma los datos para que tengan una media de 0 y una desviación estándar de 1.

print("\nEscalando los datos para la Red Neuronal...")
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
print("Datos escalados correctamente.")


Escalando los datos para la Red Neuronal...
Datos escalados correctamente.


## PASO 6: ENTRENAR Y EVALUAR MODELO 1: RANDOM FOREST

In [ ]:
# ¡La parte divertida! Aquí es donde el algoritmo aprende.
# Usaremos un 'RandomForestClassifier', que es como un comité de "árboles de decisión" que votan
# para decidir la calidad del vino. Es un modelo muy potente y versátil.

print("\nIniciando el entrenamiento del modelo RandomForest...")

# Inicializamos el modelo
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

# Entrenamos el modelo con los datos de entrenamiento (no necesitan estar escalados)
rf_model.fit(X_train, y_train)

print("¡Modelo RandomForest entrenado exitosamente!")

# Evaluamos el rendimiento
print("\nEvaluando el modelo RandomForest...")
rf_predictions = rf_model.predict(X_test)
rf_accuracy = accuracy_score(y_test, rf_predictions)
print(f"\nPrecisión de RandomForest: {rf_accuracy:.2%}")
print("\nReporte de Clasificación para RandomForest:")
print(classification_report(y_test, rf_predictions))


Iniciando el entrenamiento del modelo RandomForest...
¡Modelo RandomForest entrenado exitosamente!

Evaluando el modelo RandomForest...

Precisión de RandomForest: 82.31%

Reporte de Clasificación para RandomForest:
              precision    recall  f1-score   support

        alta       1.00      0.47      0.64        40
        baja       0.79      0.77      0.78       477
       media       0.84      0.87      0.86       783

    accuracy                           0.82      1300
   macro avg       0.88      0.71      0.76      1300
weighted avg       0.83      0.82      0.82      1300



## PASO 7: ENTRENAR Y EVALUAR MODELO 2: GRADIENT BOOSTING

In [ ]:
# Gradient Boosting es otro modelo de ensamble, como Random Forest, pero construye
# los árboles de forma secuencial, donde cada nuevo árbol intenta corregir los errores del anterior.

print("\nIniciando el entrenamiento del modelo Gradient Boosting...")
gb_model = GradientBoostingClassifier(n_estimators=100, random_state=42)
gb_model.fit(X_train, y_train)
print("¡Modelo Gradient Boosting entrenado exitosamente!")

print("\nEvaluando el modelo Gradient Boosting...")
gb_predictions = gb_model.predict(X_test)
gb_accuracy = accuracy_score(y_test, gb_predictions)
print(f"\nPrecisión de Gradient Boosting: {gb_accuracy:.2%}")
print("\nReporte de Clasificación para Gradient Boosting:")
print(classification_report(y_test, gb_predictions))


Iniciando el entrenamiento del modelo Gradient Boosting...
¡Modelo Gradient Boosting entrenado exitosamente!

Evaluando el modelo Gradient Boosting...

Precisión de Gradient Boosting: 76.31%

Reporte de Clasificación para Gradient Boosting:
              precision    recall  f1-score   support

        alta       0.75      0.15      0.25        40
        baja       0.74      0.67      0.70       477
       media       0.78      0.85      0.81       783

    accuracy                           0.76      1300
   macro avg       0.75      0.56      0.59      1300
weighted avg       0.76      0.76      0.75      1300



## PASO 8: ENTRENAR Y EVALUAR MODELO 3: RED NEURONAL (MLP)

In [ ]:
# Usaremos un Perceptrón Multicapa (MLP), un tipo de red neuronal.
# ¡IMPORTANTE! Las redes neuronales DEBEN usar los datos escalados que preparamos en el paso 5.1.

print("\nIniciando el entrenamiento del modelo de Red Neuronal (MLP)...")
# Definimos la arquitectura: 2 capas ocultas con 50 neuronas cada una.
nn_model = MLPClassifier(hidden_layer_sizes=(50, 50), max_iter=500, random_state=42)
nn_model.fit(X_train_scaled, y_train)
print("¡Modelo de Red Neuronal entrenado exitosamente!")

print("\nEvaluando el modelo de Red Neuronal...")
nn_predictions = nn_model.predict(X_test_scaled)
nn_accuracy = accuracy_score(y_test, nn_predictions)
print(f"\nPrecisión de la Red Neuronal: {nn_accuracy:.2%}")
print("\nReporte de Clasificación para la Red Neuronal:")
print(classification_report(y_test, nn_predictions))


Iniciando el entrenamiento del modelo de Red Neuronal (MLP)...
¡Modelo de Red Neuronal entrenado exitosamente!

Evaluando el modelo de Red Neuronal...

Precisión de la Red Neuronal: 75.46%

Reporte de Clasificación para la Red Neuronal:
              precision    recall  f1-score   support

        alta       0.60      0.23      0.33        40
        baja       0.70      0.71      0.71       477
       media       0.79      0.81      0.80       783

    accuracy                           0.75      1300
   macro avg       0.70      0.58      0.61      1300
weighted avg       0.75      0.75      0.75      1300



/usr/local/lib/python3.12/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:781: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


## PASO 9: GUARDAR TODOS LOS MODELOS ENTRENADOS

In [ ]:
# ¡Felicidades! Ya tienes tus modelos entrenados.
# Ahora guardaremos cada uno de nuestros modelos y el escalador en archivos separados.
# Estos archivos son los "cerebros" listos para ser usados por nuestra API.

# Guardar Modelo 1: Random Forest
rf_model_filename = 'wine_quality_rf_model.joblib'
joblib.dump(rf_model, rf_model_filename)
print(f"\nModelo Random Forest guardado en: '{rf_model_filename}'")

# Guardar Modelo 2: Gradient Boosting
gb_model_filename = 'wine_quality_gb_model.joblib'
joblib.dump(gb_model, gb_model_filename)
print(f"Modelo Gradient Boosting guardado en: '{gb_model_filename}'")

# Guardar Modelo 3: Red Neuronal
nn_model_filename = 'wine_quality_nn_model.joblib'
joblib.dump(nn_model, nn_model_filename)
print(f"Modelo de Red Neuronal guardado en: '{nn_model_filename}'")

# ¡MUY IMPORTANTE! Guardar también el escalador.
# Lo necesitaremos para escalar cualquier dato nuevo antes de pasarlo a la red neuronal.
scaler_filename = 'scaler.joblib'
joblib.dump(scaler, scaler_filename)
print(f"Escalador guardado en: '{scaler_filename}'")

print("\nProceso completado. Ya puedes descargar los archivos .joblib desde el panel de Colab.")


Modelo Random Forest guardado en: 'wine_quality_rf_model.joblib'
Modelo Gradient Boosting guardado en: 'wine_quality_gb_model.joblib'
Modelo de Red Neuronal guardado en: 'wine_quality_nn_model.joblib'
Escalador guardado en: 'scaler.joblib'

Proceso completado. Ya puedes descargar los archivos .joblib desde el panel de Colab.
